# Data Cleaning

Une fois qu'on a identifié les problèmes dans nos données, on procède à leur nettoyage.

Cela consiste en :
- la suppression des données dupliquées
- le traitement des valeurs manquantes
- la suppression des valeurs aberrantes

Cette étape est cruciale pour garantir la qualité et la fiabilité des données avant de les utiliser pour l'analyse et la modélisation.

## Import

In [20]:
import IPython.display as ipd
from scipy.signal import hilbert
import numpy as np
from scipy.stats import skew
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

import numpy as np
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed

# change style
plt.style.use('ggplot')
import librosa
import librosa.display
from tqdm import tqdm
from tools import play_audio, load_audio_file, pad_signal

tqdm.pandas()
import os
from glob import glob
import random
from datetime import datetime

from concurrent.futures import ThreadPoolExecutor, as_completed

In [21]:
from params import SOUNDS_DATASET_PATH, SAMPLE_RATE, CLASS_COLORS
from tools import play_audio, load_audio_file, pad_signal


## Load Dataset

In [22]:
dataset_csv_path = os.path.join(SOUNDS_DATASET_PATH, 'dataset_features_extracted.csv')
print("Dataset path: ", dataset_csv_path)
df_drums = pd.read_csv(dataset_csv_path)

df_drums = df_drums

# set index to file_path
df_drums.set_index('file_path', inplace=True)
print("Dataset shape: ", df_drums.shape)
df_drums

Dataset path:  G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\dataset_features_extracted.csv
Dataset shape:  (10305, 100)


,file_name,file_extension,class,split,duration,log_sum_rms,log_mean_rms,log_max_rms,rms_log_std,rms_log_diff_mean,...,spec_cent_mean,spec_cent_std,spec_bw_mean,spec_bw_std,spec_flatness_mean,spec_flatness_std,spec_rolloff_mean,spec_rolloff_std,spec_contrast_mean,spec_contrast_std
file_path,,,,,,,,,,,,,,,,,,,,,
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Conga\KAMAL - Conga (1).wav,KAMAL - Conga (1),.wav,Conga,train,0.221905,-36.586320,-1.829316,-0.739824,1.237576,0.269076,...,2771.188810,2460.565926,3513.523659,1573.305322,0.011382,0.027191,322.998047,135.162666,16.910134,6.121380
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Conga\KAMAL - Conga (2).wav,KAMAL - Conga (2),.wav,Conga,test,0.075170,-5.535370,-0.790767,-0.269122,0.479259,0.253621,...,5363.585376,1400.401081,6281.286529,869.637190,0.006250,0.002458,461.425781,25.366763,13.454006,5.019219
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Conga\KAMAL - Conga (3).wav,KAMAL - Conga (3),.wav,Conga,train,0.086417,-6.860503,-0.857563,-0.265424,0.504127,0.226923,...,1688.612761,562.744633,2684.366488,708.933286,0.000881,0.001732,406.439209,12.908702,15.434837,6.394566
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Conga\KAMAL - Conga (4).wav,KAMAL - Conga (4),.wav,Conga,train,0.063810,-9.422018,-1.570336,-0.670527,0.721422,0.379499,...,3074.094357,675.055216,3106.637318,440.107261,0.005416,0.005467,509.619141,152.600726,15.496504,6.475287
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Conga\KAMAL - Conga (5).wav,KAMAL - Conga (5),.wav,Conga,train,0.258231,-23.961166,-1.041790,-0.465791,0.610694,0.136201,...,3994.841115,1705.392297,4126.019895,665.609861,0.001800,0.003827,197.543733,17.565161,20.689341,9.458610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Shaker\Mid Seed Shaker 9.aif,Mid Seed Shaker 9,.aif,Shaker,train,0.672880,-172.912320,-2.981247,-0.822484,1.529523,0.173720,...,8552.850170,1995.923836,4688.603735,1248.938149,0.154277,0.100315,3548.523370,1844.713284,13.433769,5.270034
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Shaker\Mid Seed Shaker 10.aif,Mid Seed Shaker 10,.aif,Shaker,train,0.929456,-273.343200,-3.374607,-0.728820,1.568116,0.125416,...,9341.562555,2284.763279,5029.911848,1418.301615,0.202218,0.129527,3778.412543,2690.802937,13.046287,5.105602
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Shaker\Mid Seed Shaker 11.aif,Mid Seed Shaker 11,.aif,Shaker,train,0.875011,-244.603880,-3.218472,-0.734002,1.524488,0.125789,...,9270.489384,1901.198482,5027.825233,1365.491043,0.198355,0.105156,3781.343801,2479.147790,12.827034,5.013560


In [23]:
df_drums.info(verbose=2)

<class 'pandas.core.frame.DataFrame'>
Index: 10305 entries, G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Conga\KAMAL - Conga (1).wav to G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Shaker\Mid Seed Shaker 13.aif
Data columns (total 100 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   file_name           10305 non-null  object 
 1   file_extension      10305 non-null  object 
 2   class               10305 non-null  object 
 3   split               10305 non-null  object 
 4   duration            10300 non-null  float64
 5   log_sum_rms         10300 non-null  float64
 6   log_mean_rms        10300 non-null  float64
 7   log_max_rms         10300 non-null  float64
 8   rms_log_std         10300 non-null  float64
 9   rms_log_diff_mean   10300 non-null  float64
 10  log_crest_factor    10300 non-null  float64
 11  zcr_sum             10300 non-null  float64
 12  zcr_mean          

In [24]:
df_drums.describe().T

,count,mean,std,min,25%,50%,75%,max
duration,10300.0,1.505497,2.334286,4.643991e-02,0.295669,0.600317,1.699575,45.354331
log_sum_rms,10300.0,-335.899485,703.517150,-1.970231e+04,-314.865220,-102.666063,-38.765468,19.588580
log_mean_rms,10300.0,-2.081146,1.048484,-1.622318e+01,-2.701789,-1.947092,-1.310723,0.230454
log_max_rms,10300.0,-0.461928,0.252767,-2.434703e+00,-0.610201,-0.461931,-0.254773,0.436991
rms_log_std,10300.0,1.030937,0.560313,2.827795e-02,0.659551,0.929494,1.304707,9.146350
...,...,...,...,...,...,...,...,...
spec_flatness_std,10300.0,0.087381,0.123430,5.068475e-11,0.004067,0.026495,0.129138,0.500000
spec_rolloff_mean,10300.0,1349.102279,1648.331682,6.258669e-01,101.883765,742.100830,1975.289346,13330.109579
spec_rolloff_std,10300.0,556.483660,814.097824,0.000000e+00,50.563365,272.314390,800.815404,8177.052036
spec_contrast_mean,10300.0,15.721801,2.577595,7.243022e+00,14.115603,15.236456,16.752756,31.094175


### Keep only float columns

In [25]:
# get columns names with float type use .info()
columns_float = [k for k, v in df_drums.dtypes.to_dict().items() if v == 'float64' or v == 'int64']
columns_float

['duration',
 'log_sum_rms',
 'log_mean_rms',
 'log_max_rms',
 'rms_log_std',
 'rms_log_diff_mean',
 'log_crest_factor',
 'zcr_sum',
 'zcr_mean',
 'zcr_std',
 'zcr_loudest_frame',
 'temp_cent',
 'attack_time',
 'pitch',
 'mfcc_mean_1',
 'mfcc_mean_2',
 'mfcc_mean_3',
 'mfcc_mean_4',
 'mfcc_mean_5',
 'mfcc_mean_6',
 'mfcc_mean_7',
 'mfcc_mean_8',
 'mfcc_mean_9',
 'mfcc_mean_10',
 'mfcc_mean_11',
 'mfcc_mean_12',
 'mfcc_std_1',
 'mfcc_std_2',
 'mfcc_std_3',
 'mfcc_std_4',
 'mfcc_std_5',
 'mfcc_std_6',
 'mfcc_std_7',
 'mfcc_std_8',
 'mfcc_std_9',
 'mfcc_std_10',
 'mfcc_std_11',
 'mfcc_std_12',
 'mfcc_max_1',
 'mfcc_max_2',
 'mfcc_max_3',
 'mfcc_max_4',
 'mfcc_max_5',
 'mfcc_max_6',
 'mfcc_max_7',
 'mfcc_max_8',
 'mfcc_max_9',
 'mfcc_max_10',
 'mfcc_max_11',
 'mfcc_max_12',
 'mfcc_min_1',
 'mfcc_min_2',
 'mfcc_min_3',
 'mfcc_min_4',
 'mfcc_min_5',
 'mfcc_min_6',
 'mfcc_min_7',
 'mfcc_min_8',
 'mfcc_min_9',
 'mfcc_min_10',
 'mfcc_min_11',
 'mfcc_min_12',
 'mfcc_skew_1',
 'mfcc_skew_2',
 'mf

## Delete Full of NaN rows

In [26]:
# get rows with all NaN values
df_drums[df_drums.isna().all(axis=1)]

,file_name,file_extension,class,split,duration,log_sum_rms,log_mean_rms,log_max_rms,rms_log_std,rms_log_diff_mean,...,spec_cent_mean,spec_cent_std,spec_bw_mean,spec_bw_std,spec_flatness_mean,spec_flatness_std,spec_rolloff_mean,spec_rolloff_std,spec_contrast_mean,spec_contrast_std
file_path,,,,,,,,,,,,,,,,,,,,,


In [27]:
# delete rows with all NaN values (focus on features)
df_drums.dropna(how='all', subset=columns_float, inplace=True)
df_drums

,file_name,file_extension,class,split,duration,log_sum_rms,log_mean_rms,log_max_rms,rms_log_std,rms_log_diff_mean,...,spec_cent_mean,spec_cent_std,spec_bw_mean,spec_bw_std,spec_flatness_mean,spec_flatness_std,spec_rolloff_mean,spec_rolloff_std,spec_contrast_mean,spec_contrast_std
file_path,,,,,,,,,,,,,,,,,,,,,
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Conga\KAMAL - Conga (1).wav,KAMAL - Conga (1),.wav,Conga,train,0.221905,-36.586320,-1.829316,-0.739824,1.237576,0.269076,...,2771.188810,2460.565926,3513.523659,1573.305322,0.011382,0.027191,322.998047,135.162666,16.910134,6.121380
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Conga\KAMAL - Conga (2).wav,KAMAL - Conga (2),.wav,Conga,test,0.075170,-5.535370,-0.790767,-0.269122,0.479259,0.253621,...,5363.585376,1400.401081,6281.286529,869.637190,0.006250,0.002458,461.425781,25.366763,13.454006,5.019219
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Conga\KAMAL - Conga (3).wav,KAMAL - Conga (3),.wav,Conga,train,0.086417,-6.860503,-0.857563,-0.265424,0.504127,0.226923,...,1688.612761,562.744633,2684.366488,708.933286,0.000881,0.001732,406.439209,12.908702,15.434837,6.394566
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Conga\KAMAL - Conga (4).wav,KAMAL - Conga (4),.wav,Conga,train,0.063810,-9.422018,-1.570336,-0.670527,0.721422,0.379499,...,3074.094357,675.055216,3106.637318,440.107261,0.005416,0.005467,509.619141,152.600726,15.496504,6.475287
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Conga\KAMAL - Conga (5).wav,KAMAL - Conga (5),.wav,Conga,train,0.258231,-23.961166,-1.041790,-0.465791,0.610694,0.136201,...,3994.841115,1705.392297,4126.019895,665.609861,0.001800,0.003827,197.543733,17.565161,20.689341,9.458610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Shaker\Mid Seed Shaker 9.aif,Mid Seed Shaker 9,.aif,Shaker,train,0.672880,-172.912320,-2.981247,-0.822484,1.529523,0.173720,...,8552.850170,1995.923836,4688.603735,1248.938149,0.154277,0.100315,3548.523370,1844.713284,13.433769,5.270034
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Shaker\Mid Seed Shaker 10.aif,Mid Seed Shaker 10,.aif,Shaker,train,0.929456,-273.343200,-3.374607,-0.728820,1.568116,0.125416,...,9341.562555,2284.763279,5029.911848,1418.301615,0.202218,0.129527,3778.412543,2690.802937,13.046287,5.105602
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Shaker\Mid Seed Shaker 11.aif,Mid Seed Shaker 11,.aif,Shaker,train,0.875011,-244.603880,-3.218472,-0.734002,1.524488,0.125789,...,9270.489384,1901.198482,5027.825233,1365.491043,0.198355,0.105156,3781.343801,2479.147790,12.827034,5.013560


## Duplicates
Suppression des duplicats

### Duplicates rows

In [28]:
print(f"Nombre de lignes totalement dupliquées : {df_drums.duplicated().sum()}")

Nombre de lignes totalement dupliquées : 0


In [29]:
duplicated_focus_on_features = df_drums.duplicated(subset=columns_float)
print(
    f"Nombre de lignes dupliquées (focus on features) : {duplicated_focus_on_features.sum()} lignes (qu'on peut potentiellement supprimer)")

# print per class
df_drums[duplicated_focus_on_features].groupby("class").count()["file_name"].sort_values(ascending=False)

Nombre de lignes dupliquées (focus on features) : 238 lignes (qu'on peut potentiellement supprimer)


class
Bell                   102
808                     44
Clap                    25
Crash (Cymbal)          15
Triangle                15
Snap                    10
Kick                     9
Hat Open (Cymbal)        8
Shaker                   7
Hat Closed (Cymbal)      2
Conga                    1
Name: file_name, dtype: int64

In [30]:
duplicated_focus_on_features = df_drums.duplicated(subset=columns_float, keep=False)
# Afficher les lignes dupliquées (toutes les copies)
duplicates_df = df_drums[duplicated_focus_on_features].sort_values(by=columns_float)
duplicates_df

,file_name,file_extension,class,split,duration,log_sum_rms,log_mean_rms,log_max_rms,rms_log_std,rms_log_diff_mean,...,spec_cent_mean,spec_cent_std,spec_bw_mean,spec_bw_std,spec_flatness_mean,spec_flatness_std,spec_rolloff_mean,spec_rolloff_std,spec_contrast_mean,spec_contrast_std
file_path,,,,,,,,,,,,,,,,,,,,,
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Clap\Clap 0872.flac,Clap 0872,.flac,Clap,train,0.046440,-4.019799,-0.803960,-0.643738,0.162203,0.158863,...,4626.470339,426.147392,4128.750167,47.113411,0.045543,0.013888,1016.367188,125.854158,11.954943,4.722839
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Clap\Clap 0875.flac,Clap 0875,.flac,Clap,train,0.046440,-4.019799,-0.803960,-0.643738,0.162203,0.158863,...,4626.470339,426.147392,4128.750167,47.113411,0.045543,0.013888,1016.367188,125.854158,11.954943,4.722839
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Shaker\Mun Roe ( DB ) Shaker 1.wav,Mun Roe ( DB ) Shaker 1,.wav,Shaker,train,0.080000,-5.809393,-0.829913,-0.584962,0.225188,0.180874,...,5933.449028,45.388188,3120.448791,69.453728,0.018186,0.002066,3294.580078,82.999655,16.146953,8.335257
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Shaker\Shaker 4 (3).wav,Shaker 4 (3),.wav,Shaker,train,0.080000,-5.809393,-0.829913,-0.584962,0.225188,0.180874,...,5933.449028,45.388188,3120.448791,69.453728,0.018186,0.002066,3294.580078,82.999655,16.146953,8.335257
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Shaker\Shaker (84).wav,Shaker (84),.wav,Shaker,train,0.089456,-13.053484,-1.631685,-1.033871,0.477110,0.322956,...,11033.629294,1655.514185,4920.181711,451.140152,0.150005,0.045579,5151.818848,1894.669516,15.115996,7.899844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Triangle\STEEZE TRIANGLE 1.wav,STEEZE TRIANGLE 1,.wav,Triangle,train,8.432018,-3041.128200,-4.183120,-1.158522,0.526799,0.018816,...,15631.950113,1692.451759,4727.804466,537.949495,0.062326,0.020171,10972.455430,2412.069283,17.030874,13.472127
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Bell\Bell 02.wav,Bell 02,.wav,Bell,train,10.102041,-2368.555400,-2.719352,-0.553327,1.333677,0.041081,...,698.902792,108.795627,609.081676,160.199324,0.000689,0.004254,144.527102,76.609446,28.551427,9.587233
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Bell\[BELLS] Nxsty UK Drill Kit V2 (1).wav,[BELLS] Nxsty UK Drill Kit V2 (1),.wav,Bell,train,10.102041,-2368.555400,-2.719352,-0.553327,1.333677,0.041081,...,698.902792,108.795627,609.081676,160.199324,0.000689,0.004254,144.527102,76.609446,28.551427,9.587233


In [31]:

# Regroupez les lignes en double en fonction de leurs valeurs de features
grouped_duplicates = duplicates_df.groupby(columns_float)

# Créez une liste contenant des listes de file_paths pour chaque groupe de duplicatas
duplicate_groups = []
for _, group in grouped_duplicates:
    duplicate_groups.append(list(group.index))

# Affichez les groupes de duplicatas
for i, group in enumerate(duplicate_groups):
    print(f"# Duplicate Group {i + 1}:")
    for file_path in group:
        print(f"  - {file_path}")
    print()

# Duplicate Group 1:
  - G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Clap\Clap 0872.flac
  - G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Clap\Clap 0875.flac

# Duplicate Group 2:
  - G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Shaker\Mun Roe ( DB ) Shaker 1.wav
  - G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Shaker\Shaker 4 (3).wav

# Duplicate Group 3:
  - G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Shaker\Shaker (84).wav
  - G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Shaker\Shaker (85).wav

# Duplicate Group 4:
  - G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Shaker\SHAKER (2).wav
  - G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Shaker\CNN - Shaker.wav

# Duplicate Group 5:
  - G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Snap\CNN - Snap 05.wav
  - G:\Shared drives\PFE - IN

maintenant qu'on a les lignes dupliquées, on va les supprimer

#### Delete duplicates in dataset

In [32]:
duplicates_file_to_delete = []
file_to_delete_num_group_map = {}

# Parcourez chaque groupe de doublons
for num_group, group in enumerate(duplicate_groups, start=1):
    # Triez les fichiers audio du groupe par la taille de leur nom de fichier
    file_name_len = lambda file_path: len(os.path.basename(file_path))
    sorted_group = sorted(group, key=file_name_len)

    # Gardez le fichier audio avec le plus petit nom de fichier (le premier de la liste triée)
    to_keep = sorted_group[0]

    # Ajoutez les autres fichiers audio du groupe à la liste des fichiers à supprimer
    duplicates_file_to_delete.extend(sorted_group[1:])

    for file_path in sorted_group[1:]:
        file_to_delete_num_group_map[file_path] = num_group

print(len(set(duplicates_file_to_delete)))


238


In [33]:
# Copiez votre DataFrame pour ne pas modifier l'original
cleaned_df_drums = df_drums.copy()
# Supprimez les autres fichiers audio du groupe de doublons du DataFrame
cleaned_df_drums = cleaned_df_drums.drop(file_path for file_path in duplicates_file_to_delete)

# cleaned_df contient maintenant les données sans les doublons indésirables
print(
    f"Nombre de lignes dupliquées (focus on features) : {cleaned_df_drums.duplicated(subset=columns_float).sum()} lignes (qu'on peut potentiellement supprimer)")
cleaned_df_drums

Nombre de lignes dupliquées (focus on features) : 0 lignes (qu'on peut potentiellement supprimer)


,file_name,file_extension,class,split,duration,log_sum_rms,log_mean_rms,log_max_rms,rms_log_std,rms_log_diff_mean,...,spec_cent_mean,spec_cent_std,spec_bw_mean,spec_bw_std,spec_flatness_mean,spec_flatness_std,spec_rolloff_mean,spec_rolloff_std,spec_contrast_mean,spec_contrast_std
file_path,,,,,,,,,,,,,,,,,,,,,
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Conga\KAMAL - Conga (1).wav,KAMAL - Conga (1),.wav,Conga,train,0.221905,-36.586320,-1.829316,-0.739824,1.237576,0.269076,...,2771.188810,2460.565926,3513.523659,1573.305322,0.011382,0.027191,322.998047,135.162666,16.910134,6.121380
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Conga\KAMAL - Conga (2).wav,KAMAL - Conga (2),.wav,Conga,test,0.075170,-5.535370,-0.790767,-0.269122,0.479259,0.253621,...,5363.585376,1400.401081,6281.286529,869.637190,0.006250,0.002458,461.425781,25.366763,13.454006,5.019219
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Conga\KAMAL - Conga (3).wav,KAMAL - Conga (3),.wav,Conga,train,0.086417,-6.860503,-0.857563,-0.265424,0.504127,0.226923,...,1688.612761,562.744633,2684.366488,708.933286,0.000881,0.001732,406.439209,12.908702,15.434837,6.394566
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Conga\KAMAL - Conga (4).wav,KAMAL - Conga (4),.wav,Conga,train,0.063810,-9.422018,-1.570336,-0.670527,0.721422,0.379499,...,3074.094357,675.055216,3106.637318,440.107261,0.005416,0.005467,509.619141,152.600726,15.496504,6.475287
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Conga\KAMAL - Conga (5).wav,KAMAL - Conga (5),.wav,Conga,train,0.258231,-23.961166,-1.041790,-0.465791,0.610694,0.136201,...,3994.841115,1705.392297,4126.019895,665.609861,0.001800,0.003827,197.543733,17.565161,20.689341,9.458610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Shaker\Mid Seed Shaker 9.aif,Mid Seed Shaker 9,.aif,Shaker,train,0.672880,-172.912320,-2.981247,-0.822484,1.529523,0.173720,...,8552.850170,1995.923836,4688.603735,1248.938149,0.154277,0.100315,3548.523370,1844.713284,13.433769,5.270034
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Shaker\Mid Seed Shaker 10.aif,Mid Seed Shaker 10,.aif,Shaker,train,0.929456,-273.343200,-3.374607,-0.728820,1.568116,0.125416,...,9341.562555,2284.763279,5029.911848,1418.301615,0.202218,0.129527,3778.412543,2690.802937,13.046287,5.105602
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Shaker\Mid Seed Shaker 11.aif,Mid Seed Shaker 11,.aif,Shaker,train,0.875011,-244.603880,-3.218472,-0.734002,1.524488,0.125789,...,9270.489384,1901.198482,5027.825233,1365.491043,0.198355,0.105156,3781.343801,2479.147790,12.827034,5.013560


sauvegarder dans un fichier csv les données dupliquées (pour les supprimer manuellement)

In [34]:
# save duplicates in txt file
with open(os.path.join(SOUNDS_DATASET_PATH, "__duplicates.txt"), "w", encoding="utf-8") as f:
    for file_path in duplicates_file_to_delete:
        f.write(f"{file_path} {file_to_delete_num_group_map[file_path]}" + os.linesep)

### Duplicates file_name

In [35]:
print(
    f"Nombre de lignes dupliquées sur la colonne 'file_name' : {cleaned_df_drums.duplicated(subset=['file_name', 'file_extension']).sum()} lignes (qu'on peut potentiellement supprimer)")

cleaned_df_drums[cleaned_df_drums.duplicated(subset=['file_name', 'file_extension'], keep=False)].sort_values(
    by=['file_name', 'file_extension'])

Nombre de lignes dupliquées sur la colonne 'file_name' : 0 lignes (qu'on peut potentiellement supprimer)


,file_name,file_extension,class,split,duration,log_sum_rms,log_mean_rms,log_max_rms,rms_log_std,rms_log_diff_mean,...,spec_cent_mean,spec_cent_std,spec_bw_mean,spec_bw_std,spec_flatness_mean,spec_flatness_std,spec_rolloff_mean,spec_rolloff_std,spec_contrast_mean,spec_contrast_std
file_path,,,,,,,,,,,,,,,,,,,,,


### Duplicates (too similar) audio content

Pour détecter les fichiers audio dupliqués,  on va comparer les caractéristiques audio de chaque fichier audio. Si les caractéristiques audio sont identiques, alors les fichiers audio sont dupliqués.

In [ ]:
# euclidean distance with np.linalg.norm
def euclidean_distance(vector1: np.ndarray, vector2: np.ndarray):
    """
    Compute euclidean distance between two vectors
    """
    return np.linalg.norm(vector1 - vector2)


# cosine similarity with np.dot
def cosine_similarity(vector1: np.ndarray, vector2: np.ndarray):
    """
    Compute cosine similarity between two vectors
    """
    return np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))


def similarity(vector1: np.ndarray, vector2: np.ndarray, metric: str = "euclidean"):
    """
    Compute similarity between two vectors using the specified metric. Use numpy functions.
    """
    if metric == "cosine":
        sim = cosine_similarity(vector1, vector2)
        return sim
        # Normalize cosine similarity to [0, 1]
        #return (sim + 1) / 2
    elif metric == "euclidean":
        dist = euclidean_distance(vector1, vector2)
        # Normalize euclidean distance to [0, 1] by dividing by the maximum possible distance
        max_dist = np.sqrt(len(vector1))
        return 1 - (dist / max_dist)
    else:
        raise ValueError(f"Unknown metric: {metric}")


def compute_similarity_pair(file_i, row_i, df_X, metric="euclidean", threshold : float = None):

    if threshold is None:
        threshold = 0 if metric == "cosine" else 0.5


    similarities = {}
    for file_j, row_j in df_X.iterrows():
        if file_i == file_j:
            continue
        if (file_i, file_j) in similarities or (file_j, file_i) in similarities:
            continue
        vectori = row_i.to_numpy()
        vectorj = row_j.to_numpy()
        sim = similarity(vectori, vectorj, metric=metric)
        if sim >= threshold:
            similarities[(file_i, file_j)] = sim

    return similarities


def compute_similarities_parallel(df_X: pd.DataFrame, metric: str = "euclidean", threshold : float = None, n_jobs: int = -1):
    similarities = {}
    #n_jobs = -1  # Utilisez tous les cœurs disponibles
    results = Parallel(n_jobs=n_jobs)(
        delayed(compute_similarity_pair)(file_i, row_i, df_X, metric=metric, threshold=threshold)
        for file_i, row_i in tqdm(df_X.iterrows(), total=len(df_X))
    )
    for r in results:
        similarities.update(r)
    return similarities


def compute_df_similarities(df_X: pd.DataFrame, metric: str = "euclidean", threshold : float = None, n_jobs: int = -1):
    similarities = compute_similarities_parallel(df_X, metric=metric, threshold=threshold, n_jobs=n_jobs)

    df_similarities = pd.DataFrame.from_dict(
        similarities, orient='index', columns=['similarity']
    ).sort_values(by=['similarity'], ascending=False)

    df_similarities.reset_index(inplace=True)
    df_similarities["file_i"] = df_similarities["index"].apply(lambda x: x[0])
    df_similarities["file_j"] = df_similarities["index"].apply(lambda x: x[1])
    df_similarities.drop(columns=["index"], inplace=True)


    return df_similarities




df_similarities = compute_df_similarities(cleaned_df_drums[columns_float], metric="cosine", threshold=0.7, n_jobs=2)
df_similarities


  1%|          | 72/10062 [00:16<40:11,  4.14it/s]

#### Similary > 0.999

In [ ]:
df_similarities_0_999 = df_similarities.query("similarity > 0.999")
df_similarities_0_999

In [ ]:
# play some audio files with similarity > 0.999
for i, row in df_similarities_0_999.sample(10).iterrows():
    print(f"#### similarity: {row['similarity']}")
    play_audio(row["file_i"])
    play_audio(row["file_j"])

#### Delete similar files

In [ ]:
# pandas display full table
#pd.set_option('display.max_rows', None)
table_loser_0_999 = pd.Series(
    df_similarities_0_999["file_i"].to_list() + df_similarities_0_999["file_j"].to_list()).value_counts()
table_loser_0_999

In [ ]:
play_audio("G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Conga\Conga (119).wav")
play_audio("G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Conga\Conga (118).wav")

In [ ]:
similar_file_to_delete = []  # list of file to delete
for i, row in df_similarities_0_999.iterrows():
    if row["file_i"] in similar_file_to_delete or row["file_j"] in similar_file_to_delete:
        continue
    if table_loser_0_999[row["file_j"]] > table_loser_0_999[row["file_i"]]:
        #print(f"{os.path.basename(row['file_j'])} lose vs. {os.path.basename(row['file_i'])} (because {table_loser_0_999[row['file_j']]} > {table_loser_0_999[row['file_i']]})")
        similar_file_to_delete.append(row["file_j"])
    else:
        #print(f"{os.path.basename(row['file_i'])} lose vs. {os.path.basename(row['file_j'])} (because {table_loser_0_999[row['file_i']]} > {table_loser_0_999[row['file_j']]})")
        similar_file_to_delete.append(row["file_i"])

len(similar_file_to_delete)

In [ ]:
# delete similar files from cleaned_df_drums
cleaned_df_drums = cleaned_df_drums.drop(similar_file_to_delete)
cleaned_df_drums

on affiche maintenant les fichiers qui ont été supprimés, pour aller les supprimer manuellement

In [ ]:
similar_file_to_delete

## Missing values Processing
Traitement des valeurs manquantes

In [ ]:
cleaned_df_drums.isnull().any()

In [ ]:
cleaned_df_drums.isnull().sum()

In [ ]:
cleaned_df_drums.describe().T

In [ ]:
cleaned_df_drums.loc

## Outliers

Un outlier (ou valeur aberrante) est une observation qui se situe à une distance anormalement grande des autres observations dans un ensemble de données. Les outliers peuvent être causés par des erreurs de mesure, des erreurs d'enregistrement, ou par des variations naturelles dans les données. Ils peuvent avoir un impact significatif sur l'analyse et la modélisation des données, en introduisant des biais et en réduisant la performance des modèles prédictifs.

Dans le contexte de notre projet, les outliers peuvent correspondre à des sons de batterie ayant des caractéristiques très différentes des autres sons, qui pourraient rendre difficile la classification ou l'analyse ultérieure.

In [ ]:
cleaned_df_drums.describe().T

In [ ]:
cleaned_df_drums["duration"].boxplot()

In [ ]:
from collections import Counter


def get_outliers_iqr_per_class(df, column, class_column, multiplier=1.5):
    outliers_indices = []

    # Divisez le dataframe en sous-groupes en fonction des classes.
    for class_value in df[class_column].unique():
        class_df = df[df[class_column] == class_value]

        # Appliquez la méthode IQR pour chaque sous-groupe.
        Q1 = class_df[column].quantile(0.25)
        Q3 = class_df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - multiplier * IQR
        upper_bound = Q3 + multiplier * IQR
        outliers_class_df = class_df[(class_df[column] < lower_bound) | (class_df[column] > upper_bound)]

        outliers_indices.extend(outliers_class_df.index.tolist())

    return outliers_indices


# Remplacez 'class_column' par le nom de la colonne contenant les classes dans votre dataframe.
class_column = 'class'

# Parcourez toutes les colonnes de cleaned_df_drums pour lesquelles vous souhaitez détecter les outliers.
outliers_counter = Counter()
for col in tqdm(columns_float):
    outliers_indices = get_outliers_iqr_per_class(cleaned_df_drums, col, class_column)
    outliers_counter.update(outliers_indices)

limit_outliers_count = 15
# Trouvez les index des lignes qui ont été détectées au moins 3 fois comme outliers.
outliers_to_remove = [index for index, count in outliers_counter.items() if count >= limit_outliers_count]
print(f"Nombre d'outliers à supprimer: {len(outliers_to_remove)}")

for file_outlier in outliers_to_remove[:10]:
    play_audio(file_outlier)

In [ ]:
# Supprimez les outliers du dataframe.
cleaned_df_drums = cleaned_df_drums.drop(outliers_to_remove)
print(f"Nombre de lignes restantes: {len(cleaned_df_drums)}")
cleaned_df_drums

## Save cleaned dataframe

In [ ]:
now_str = datetime.now().strftime("%Y%m%d")
cleaned_df_drums.to_csv(os.path.join(SOUNDS_DATASET_PATH, "dataset_cleaned_features_extracted.csv"), index=False)